In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import glob
import natsort
import tifffile as tiff


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")
 
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\PLA\HCC827 cell culture'


# 3D segmentation

In [4]:
from natsort import natsorted
import tifffile as tiff

def make_imgs_same_dim(imgs):
    # Get max dimensions
    shapes = np.array([img.shape[1:] for img in imgs])
    min_x, min_y = shapes.min(axis=0)
        
    return [img[:, :min_x, :min_y] for img in imgs]

In [5]:
df_imgs_path = data_dir / '9PPI Cell Culture' / '3D_Whole' /  'metadata' / 'imgs_reg_norm.csv'
df_imgs = pd.read_csv(df_imgs_path)

whole_seg_path = data_dir / '9PPI Cell Culture' / '3D_Whole' / 'imgs' / 'segmentation'
mask_path = data_dir / '9PPI Cell Culture' / '3D_Whole' / 'imgs' / 'masks_3D'
mask_path .mkdir(parents=True, exist_ok=True)

In [6]:
from cellpose import core, utils, io, models, metrics
use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')


>>> GPU activated? YES


In [7]:
group = df_imgs.groupby(['Timepoint', 'FOV'])
n_limit = 4

for name, df_group in group:
    exp = '_'.join(name)
    f_names = glob.glob(str(whole_seg_path)+f'\\{exp}*')
    f_names = natsorted(f_names)
    
    # Read imgs
    imgs = [skimage.io.imread(f).transpose(2, 0, 1) for f in f_names]
    imgs = np.stack(make_imgs_same_dim(imgs))
    imgs_middle = imgs[n_limit:-n_limit]
    
    # declare model
    model = models.CellposeModel(gpu=True, model_type='CP')

    # run model on test images
    masks, flows, styles = model.eval(imgs_middle, 
                                  channels=[2,3],
                                  diameter=180,
                                  flow_threshold=0.6,
                                  cellprob_threshold=-3,
                                  do_3D=True,
                                  anisotropy= 2.1, 
                                  )
    
    file_path = mask_path / f'{exp}.tif'
    tiff.imwrite(file_path, masks)
    
    # declare model
    model = models.CellposeModel(gpu=True, model_type='nuclei')
    # run model on test images
    masks, flows, styles = model.eval(imgs_middle, 
                                  channels=[3,0],
                                  diameter=100,
                                  flow_threshold=0.6,
                                  do_3D=True,
                                  anisotropy= 2.1, 
                                  )
    
    file_path = mask_path / f'Nuclei_{exp}.tif'
    tiff.imwrite(file_path, masks)

# Extract cell info

In [5]:
from skimage import morphology, measure
from skimage.segmentation import clear_border
    
def count_pixel_label_mask(regionmask, intensity_image):
    v,c = np.unique(intensity_image[regionmask], return_counts=True)
    return dict(zip(v,c))
    
# Quality control of mask
def qc_nuclei(mask_cyto, mask_nuclei, small_size=100000):
    '''
    Function to check if cell masks contain nuclei
    '''
    # Dictionnary storing nuclei and cyto label to cell id 
    nuclei2cell = {}
    cyto2cell = {}
    
    # Filter out small objects
    mask_cyto = morphology.remove_small_objects(mask_cyto,  min_size=small_size)    
    
    # Filter out mask touching border
    segmented_padded = np.pad(
        mask_cyto,
        ((1, 1), (0, 0), (0, 0)),
        mode='constant',
        constant_values=0,
    )
    mask_cyto = clear_border(segmented_padded)[1:-1]
    
    # Filtered only cell mask region
    cell_mask = np.where(mask_cyto > 0, 1, 0)
    mask_nuclei_filtered = mask_nuclei * cell_mask
    mask_nuclei_filtered =  morphology.remove_small_objects(mask_nuclei_filtered,  min_size=small_size//4)
    
    nuclei_mask = np.where(mask_nuclei>0, 1, 0)
    cyto = (mask_cyto - mask_cyto*nuclei_mask).astype(np.uint16)
    
    # Count pixel cell label in each nuclei region to assign each nuclei to cell
    props = measure.regionprops(mask_nuclei_filtered, intensity_image=mask_cyto, 
                    extra_properties=(count_pixel_label_mask,))
    nuclei_labels = []
    cell_labels = []
    for prop in props:
        df = pd.DataFrame.from_dict(prop['count_pixel_label_mask'], orient='index').reset_index()
        df.columns = ['Label', 'Count']
        corresponding_label = df.iloc[df['Count'].argmax(axis=0)]['Label']
        nuclei_labels.append(prop['Label'])
        cell_labels.append(corresponding_label)
    
    df = pd.DataFrame({'Nuclei': nuclei_labels, 'Cyto': cell_labels})
    return mask_cyto, mask_nuclei_filtered, cyto, df

In [7]:
from collections import defaultdict

data_dir = (Path().cwd().parents[0] / 'data').absolute()

# Get image dataframe 
df_imgs_path = data_dir / '9PPI Cell Culture' / '3D_Whole' / 'metadata' / 'imgs_reg_norm.csv'
df_imgs = pd.read_csv(df_imgs_path)


In [8]:
# Read masks paths 
mask_dir = data_dir / '9PPI Cell Culture' / '3D_Whole' / 'imgs' / 'masks_3D'
mask_filt_dir = data_dir / '9PPI Cell Culture' / '3D_Whole' / 'imgs' / 'masks_3D_filtered'
mask_filt_dir .mkdir(parents=True, exist_ok=True)

masks_path = defaultdict(dict) 
for path in os.listdir(mask_dir):
    name = path.split('.')[0]
    if 'Nuclei' in name:
        masks_path[name[7:]]['nuclei'] = mask_dir / path
    else:
        masks_path[name]['cyto'] = mask_dir / path

In [9]:
group = df_imgs.groupby(['Timepoint', 'FOV'])

for name, df_group in group:
    name = '_'.join(name)
    
    # Read masks
    mask_cyto_path = masks_path[name]['cyto']
    mask_nuclei_path = masks_path[name]['nuclei']
    
    mask_cyto = skimage.io.imread(mask_cyto_path)
    mask_nuclei = skimage.io.imread(mask_nuclei_path)
    
    cell, nuclei, cyto, df = qc_nuclei(mask_cyto, mask_nuclei)
    
    file_path =  mask_filt_dir / f'Nuclei_{name}.tif'
    tiff.imwrite(file_path, nuclei)
    file_path =  mask_filt_dir / f'Cell_{name}.tif'
    tiff.imwrite(file_path, cell)
    file_path =  mask_filt_dir / f'Cyto_{name}.tif'
    tiff.imwrite(file_path, cyto)
    file_path =  mask_filt_dir / f'df_{name}.csv'
    df.to_csv(file_path, index=False)


In [10]:
# import napari 

# viewer = napari.view_labels(cell)
# viewer.add_labels(nuclei)

<Labels layer 'nuclei' at 0x11a9669aac0>

# Extract cell contours

In [11]:
from skimage import measure
import cv2 

def get_contours_mask(mask):
    contours = []
    labels = []
    for l in tqdm(np.unique(mask)):
        if l == 0:
            continue
        contours.append(measure.find_contours((mask==l).astype(np.uint8), 0.8))
        labels.append(l)
    return contours, labels

def replace_with_dict2(ar, dic):
    # Extract out keys and values
    k = np.array(list(dic.keys()))
    v = np.array(list(dic.values()))

    # Get argsort indices
    sidx = k.argsort()
    
    ks = k[sidx]
    vs = v[sidx]
    return vs[np.searchsorted(ks,ar)]

In [12]:
mask_filt_dir = data_dir / '9PPI Cell Culture' / '3D_Whole' / 'imgs' / 'masks_3D_filtered'

masks_path = defaultdict(dict) 
for path in os.listdir(mask_filt_dir):
    name = path.split('.')[0]
    if 'Nuclei' in name:
        masks_path[name[7:]]['nuclei'] = mask_filt_dir / path
    elif 'Cyto' in name:
        masks_path[name[5:]]['cyto'] =mask_filt_dir / path
    elif 'Cell' in name:
        masks_path[name[5:]]['cell'] =mask_filt_dir / path    
    elif 'df' in name:
        masks_path[name[3:]]['df'] =mask_filt_dir / path
    else:
        pass

In [14]:
from collections import defaultdict

data_dir = (Path().cwd().parents[0] / 'data').absolute()

# Get image dataframe 
df_imgs_path = data_dir / '9PPI Cell Culture'/ '3D_Whole' / 'metadata' / 'imgs_reg_norm.csv'
df_imgs = pd.read_csv(df_imgs_path)


In [ ]:
contour_save_path =  data_dir / '9PPI Cell Culture' / '3D_Whole' / 'contours'
contour_save_path.mkdir(parents=True, exist_ok=True)

group = df_imgs.groupby(['Timepoint', 'FOV'])

for name, df_group in group:
    name = '_'.join(name)
    # Read masks
    mask_cyto_path = masks_path[name]['cell']
    mask_nuclei_path = masks_path[name]['nuclei']
    df_path =  masks_path[name]['df']
    
    mask_cyto = skimage.io.imread(mask_cyto_path)
    mask_nuclei = skimage.io.imread(mask_nuclei_path)
    df_cell_info = pd.read_csv(df_path)
    nuclei2cell = dict(zip(df_cell_info.iloc[:,0], df_cell_info.iloc[:,1])) 
    nuclei2cell[0] = 0
    replaced = replace_with_dict2(mask_nuclei, nuclei2cell)
    
    contours_dict = {}
    for z in range(len(mask_cyto)):
        # Extract contours
        contours_cyto, labels_cyto = get_contours_mask(mask_cyto[z])
        contours_nuclei, labels_nuclei = get_contours_mask(replaced[z])
        contours_dict[z] = [contours_cyto, labels_cyto, contours_nuclei, labels_nuclei]
    
    # Save data
    save_path = contour_save_path / (name+'.pkl')
    with open(save_path, 'wb') as f:
        pickle.dump([contours_cyto, labels_cyto, contours_nuclei, labels_nuclei], 
                    f, protocol=pickle.HIGHEST_PROTOCOL)    


  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/316 [00:00<?, ?it/s]

  0%|          | 0/334 [00:00<?, ?it/s]

  0%|          | 0/316 [00:00<?, ?it/s]

  0%|          | 0/335 [00:00<?, ?it/s]

...

  0%|          | 0/334 [00:00<?, ?it/s]

  0%|          | 0/316 [00:00<?, ?it/s]

  0%|          | 0/335 [00:00<?, ?it/s]

  0%|          | 0/316 [00:00<?, ?it/s]

  0%|          | 0/337 [00:00<?, ?it/s]

# 3D PPI

In [31]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

df_meta_path = data_dir / '9PPI Cell Culture' / '3D_Whole' / 'metadata' / 'imgs_reg_norm.csv'
df_imgs = pd.read_csv(df_meta_path)

In [32]:
import PLA

PPI_save_path =  data_dir / '9PPI Cell Culture' / '3D_Whole'  / 'PPI'
PPI_save_path.mkdir(parents=True, exist_ok=True)

PPI_img_path = data_dir /'9PPI Cell Culture' / '3D_Whole'  / 'imgs' / 'imgs_PPI'

In [ ]:
PPI_names = ['Cyclin D1 & CDK2', 'NF-Kb & p-P90rsk', 'Sox2 & Oct4', 'Bim & Tom20', 'Mcl-1 & BAK', 
             'p-ERK & c-MYC', 'TEAD1 & YAP',  'Cyclin E & CDK4', 'P-AKT & mTOR']

group = df_imgs.groupby(['Timepoint', 'FOV'])
n_limit = 5

for name, df_group in group:
    df_group = df_group.sort_values(['Z'])
    df_group = df_group.iloc[n_limit:-n_limit]
    paths = df_group.Path.tolist()
    
    pla_detect = PLA.PLA_detection_3D(paths)
    for PPI_pair in PPI_names: 
        img_spot, img_wth, spots_filtered, imgs = pla_detect.detect_spot(PPI_pair, thres=0.04)
        # fname = '_'.join(name)+f'{PPI_pair}.tiff'
        # tiff.imwrite(PPI_img_path / fname, img_wth)
        
    # Save PPI dict
    fname = '_'.join(name)+'.pkl'
    pla_detect.save_pickle(PPI_save_path / fname)

Reading images:   0%|          | 0/14 [00:00<?, ?it/s]

Detecting PPI spots:   0%|          | 0/14 [00:00<?, ?it/s]

(array([0, 1], dtype=uint8), array([29452901,   222329], dtype=int64))


Reading images:   0%|          | 0/14 [00:00<?, ?it/s]

Detecting PPI spots:   0%|          | 0/14 [00:00<?, ?it/s]

...

Reading images:   0%|          | 0/14 [00:00<?, ?it/s]

Detecting PPI spots:   0%|          | 0/14 [00:00<?, ?it/s]

(array([0, 1], dtype=uint8), array([29642086,    33144], dtype=int64))


Reading images:   0%|          | 0/14 [00:00<?, ?it/s]

Detecting PPI spots:   0%|          | 0/14 [00:00<?, ?it/s]

# Extract count 

In [5]:
import pickle 
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import scipy 

def read_PPI(path):
    with open(path, 'rb') as file:
        PPI_dict = pickle.load(file)

    return PPI_dict

def get_NN_radius(data, r):
    fit = NearestNeighbors(radius=r).fit(data)
    m = fit.radius_neighbors(data, return_distance=True, sort_results=True)

    # Put in dataframe format
    neighbours = pd.DataFrame(m[1].tolist(), index = data.index)
    
    A = fit.radius_neighbors_graph(data)
    return neighbours, A

def plot_spot_on_image(spots, spacing, radius):
    spot_img = np.zeros(spots.max(0))
    coords = (spots[:, :3] / spacing).astype(int)
    r = radius  # shorthand
    for coord in coords:
        slc = tuple(slice(x-r, x+r) if i != 0 else slice(x, x+1) for i, x in enumerate(coord) )
        spot_img[slc] = 1
    return spot_img

def plot_label_on_image(spots, spacing, radius, labels):
    spot_img = np.zeros(spots.max(0))
    coords = (spots[:, :3] / spacing).astype(int)
    r = radius  # shorthand
    for i, coord in enumerate(coords):
        slc = tuple(slice(x-r, x+r) if i != 0 else slice(x, x+1) for i, x in enumerate(coord) )
        spot_img[slc] = labels[i]
    return spot_img

def create_PPI_df(df_loc_filtered, mask_cyto, mask_nuclei, name):
    _, y_max, x_max = mask_cyto.shape
    z, y, x = df_loc_filtered['z'].to_numpy(), df_loc_filtered['y'].to_numpy(), df_loc_filtered['x'].to_numpy()
    y = np.clip(y, a_min=0, a_max=y_max-1)
    x = np.clip(x, a_min=0, a_max=x_max-1)

    labels = mask_cyto[z, y, x]
    labels_nuclei = mask_nuclei[z, y, x].astype(np.uint8)
    
    df_per_cell = pd.DataFrame({'Cell': labels, 'Nuclei': labels_nuclei, 'z': z, 'y':y, 'x':x})
    df_per_cell['PPI'] = name
    return df_per_cell

In [6]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

df_meta_path = data_dir / '9PPI Cell Culture' / '3D_Whole' / 'metadata' / 'imgs_reg_norm.csv'
df_imgs = pd.read_csv(df_meta_path)

PPI_save_path =  data_dir / '9PPI Cell Culture' / '3D_Whole'  / 'PPI'

In [7]:
mask_filt_dir = data_dir /'9PPI Cell Culture'/ '3D_Whole' / 'imgs' / 'masks_3D_filtered'

masks_path = defaultdict(dict) 
for path in os.listdir(mask_filt_dir):
    name = path.split('.')[0]
    if 'Nuclei' in name:
        masks_path[name[7:]]['nuclei'] = mask_filt_dir / path
    elif 'Cyto' in name:
        masks_path[name[5:]]['cyto'] =mask_filt_dir / path
    elif 'Cell' in name:
        masks_path[name[5:]]['cell'] =mask_filt_dir / path    
    elif 'df' in name:
        masks_path[name[3:]]['df'] =mask_filt_dir / path
    else:
        pass

In [8]:
PPI_save_path =  data_dir / '9PPI Cell Culture' / '3D_Whole' / 'PPI'
group = df_imgs.groupby(['Timepoint', 'FOV'])

for names, df_group in group:
    name = '_'.join(names)
    # Read PPi
    PPI_dict = read_PPI(PPI_save_path / f'{name}.pkl')
    
    # Read mask
    mask_cyto_path = masks_path[name]['cell']
    mask_nuclei_path = masks_path[name]['nuclei']
    df_path =  masks_path[name]['df']
    
    mask_cyto = skimage.io.imread(mask_cyto_path)
    mask_nuclei = skimage.io.imread(mask_nuclei_path)
    _, y_max, x_max = mask_cyto.shape

    df_PPIs = []
    for k in ['Cyclin D1 & CDK2', 'NF-Kb & p-P90rsk', 'Sox2 & Oct4', 'Bim & Tom20', 'Mcl-1 & BAK', 
             'p-ERK & c-MYC', 'TEAD1 & YAP',  'Cyclin E & CDK4', 'P-AKT & mTOR']:
        PPI_loc = PPI_dict[k][:, :3].astype(np.uint32)
        
        # Get PPI loc in panda dataframe format
        df_loc = pd.DataFrame(PPI_loc, columns=['z', 'y', 'x'])

        # Get dot neighboring graph with user defined radius
        nei, A = get_NN_radius(df_loc[['y', 'x']], r=2.5)

        # Assign new labels based on connected components label
        labels = scipy.sparse.csgraph.connected_components(A, directed=False)[1]
        df_loc['CC_label'] = labels

        # Group by CC and extract mean position
        df_loc_filtered = df_loc.groupby(['CC_label']).mean().astype(np.uint32)
        
        # Extract Cell info per PPI
        df_PPI = create_PPI_df(df_loc_filtered, mask_cyto, mask_nuclei, k)
        df_PPIs.append(df_PPI)
    df_PPI = pd.concat(df_PPIs)
    df_PPI['Condition'] = names[0]
    df_PPI['FOV'] = names[1]
    
    # Save dataframe
    path = PPI_save_path / f'{name}.csv'
    df_PPI.to_csv(path, index=False)

In [ ]:
# img_spot = plot_label_on_image(df_loc_filtered.values, 1, 1, labels)
# img_spot = img_spot.astype(np.uint16)

In [ ]:
# import napari

# viewer = napari.view_labels(img_spot)
# viewer.add_labels(mask_cyto)

# Generate PPI network

In [9]:
from sklearn.preprocessing import OneHotEncoder
import networkx as nx
from sklearn import preprocessing
import scipy 
from scipy.spatial import Delaunay
import itertools
import pickle

def plot_tri_simple(ax, points, tri):
    for tr in tri.simplices:
        pts = points[tr, :]
        ax.plot3D(pts[[0,1],0], pts[[0,1],1], pts[[0,1],2], color='g', lw='0.1')
        ax.plot3D(pts[[0,2],0], pts[[0,2],1], pts[[0,2],2], color='g', lw='0.1')
        ax.plot3D(pts[[0,3],0], pts[[0,3],1], pts[[0,3],2], color='g', lw='0.1')
        ax.plot3D(pts[[1,2],0], pts[[1,2],1], pts[[1,2],2], color='g', lw='0.1')
        ax.plot3D(pts[[1,3],0], pts[[1,3],1], pts[[1,3],2], color='g', lw='0.1')
        ax.plot3D(pts[[2,3],0], pts[[2,3],1], pts[[2,3],2], color='g', lw='0.1')

    ax.scatter(points[:,0], points[:,1], points[:,2], color='b')
    
def create_network(df, t=50, scale=0.2):   
    z,y,x = df['z'].to_numpy()*scale, df['y'].to_numpy(), df['x'].to_numpy()

    # Get coordinates
    coordinates = np.vstack([x, y, z]).T
    points = coordinates - np.mean(coordinates, axis=0)
    
    # Delaunay
    tri = Delaunay(points)
    G = nx.Graph()
    for path in tri.simplices:
        G.add_nodes_from(path)
        edges = list(itertools.combinations(path, 2))
        G.add_edges_from(edges)
    
    # Euclidian distance
    distance = scipy.spatial.distance_matrix(points, points)
    l = t**2/np.log(t)
    distance_norm = np.exp(-distance**2/t)

    # distance_norm = np.where(distance>t, 0, np.exp(-distance*2/l))
    
    g_dist = nx.from_numpy_array(distance_norm)
    g = g_dist.edge_subgraph(G.edges()).copy() # Only keep delaunay edges
    nx.set_node_attributes(g, dict(zip(g.nodes(), df.Labels)), "labels")
    nuclei = (df.Nuclei > 0).astype(int).tolist()
    nx.set_node_attributes(g, dict(zip(g.nodes(), nuclei)), "nuclei")
    nx.set_node_attributes(g, dict(zip(g.nodes(), coordinates)), "pos")
    return g

In [10]:
PPI_save_path =  data_dir / '9PPI Cell Culture' / '3D_Whole' / 'PPI'

# Read PPi info 
dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)
df = pd.concat(dfs)

# Create label 
enc = OneHotEncoder(handle_unknown='ignore')
labels = enc.fit_transform(df['PPI'].to_numpy().reshape(-1, 1)).toarray().astype(np.uint8)
df['Labels'] = labels.tolist()

In [11]:
df

,Cell,Nuclei,z,y,x,PPI,Condition,FOV,Labels
0,1015,0,2,1547,2471,Cyclin D1 & CDK2,HCC827Ctrl,FW1,"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
1,1178,0,4,1796,6060,Cyclin D1 & CDK2,HCC827Ctrl,FW1,"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
2,138,0,1,468,6123,Cyclin D1 & CDK2,HCC827Ctrl,FW1,"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
3,138,0,6,343,6106,Cyclin D1 & CDK2,HCC827Ctrl,FW1,"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,1639,0,3,2794,3508,Cyclin D1 & CDK2,HCC827Ctrl,FW1,"[0, 1, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...
84422,0,0,13,2344,7460,P-AKT & mTOR,HCC827Osim,FW2,"[0, 0, 0, 0, 0, 1, 0, 0, 0]"
84423,390,176,13,2730,5373,P-AKT & mTOR,HCC827Osim,FW2,"[0, 0, 0, 0, 0, 1, 0, 0, 0]"
84424,0,0,13,3137,5453,P-AKT & mTOR,HCC827Osim,FW2,"[0, 0, 0, 0, 0, 1, 0, 0, 0]"
84425,501,4,13,3044,1995,P-AKT & mTOR,HCC827Osim,FW2,"[0, 0, 0, 0, 0, 1, 0, 0, 0]"


In [12]:
graph_save_path =  data_dir / '9PPI Cell Culture' / '3D_Whole' / 'graphs' / 'raw'
graph_save_path .mkdir(parents=True, exist_ok=True)

scale = 0.4/0.18872

group = df.groupby(['Condition', 'FOV', 'Cell'])
for name, df_group in group:
    if name[2] == 0:
        continue
    if len(df_group) < 5:
        continue
    
    g = create_network(df_group, t=70, scale=scale)
    if nx.number_of_isolates(g)>0:
        print('Isolated nodes')
    
    # Save pgrahs
    save_path = graph_save_path / ('_'.join([str(n) for n in name])+'.pkl')
    with open(save_path, 'wb') as f:
        pickle.dump(g, f, protocol=pickle.HIGHEST_PROTOCOL)


In [12]:
# scale = 0.4/0.18872
# z,y,x = df_group['z'].to_numpy()*scale, df_group['y'].to_numpy(), df_group['x'].to_numpy()

# points = np.vstack([x, y, z]).T
# tri = Delaunay(points)

# G = nx.Graph()
# for path in tri.simplices:
#     G.add_nodes_from(path)
#     edges = list(itertools.combinations(path, 2))
#     G.add_edges_from(edges)
    
# pos = points
# node_xyz = np.array([pos[v] for v in sorted(G)])
# edge_xyz = np.array([(pos[u], pos[v]) for u, v in G.edges()])

# # Create the 3D figure
# fig = plt.figure()
# ax = fig.add_subplot(111, projection="3d")

# # Plot the nodes - alpha is scaled by "depth" automatically
# ax.scatter(*node_xyz.T, color='b')
# for vizedge in edge_xyz:
#     ax.plot(*vizedge.T, color='g', lw=0.5)

In [13]:
# fig = plt.figure()
# ax = plt.axes(projection='3d')
# plot_tri_simple(ax, points, tri)